In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [51]:
for i in range(1,13):
    if i == 1:
        df = pd.read_csv('raw_data/all_energy_statistics' + str(i) +'.csv')
    else:
        df1 = pd.read_csv('raw_data/all_energy_statistics' + str(i) + '.csv',header=None)
        df = df.append(df1) 

In [52]:
df.head()

,0,1,2,3,4,5,6,category,commodity_transaction,country_or_area,quantity,quantity_footnotes,unit,year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Exports,Austria,5.0,NaN,"Metric tons, thousand",1996.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Exports,Austria,17.0,NaN,"Metric tons, thousand",1995.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Exports,Belgium,0.0,NaN,"Metric tons, thousand",2014.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Exports,Belgium,0.0,NaN,"Metric tons, thousand",2013.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Exports,Belgium,35.0,NaN,"Metric tons, thousand",2012.0


In [53]:
for i in range(1,13):
    if i == 1:
        df2 = pd.read_csv('raw_data/all_energy_statistics' + str(i) +'.csv')
        df2_cols = df2.columns.to_list()
    else:
        df3 = pd.read_csv('raw_data/all_energy_statistics' + str(i) + '.csv',header=None)
        df3.columns = df2_cols
        df2 = df2.append(df3)  

In [54]:
df.head()

,0,1,2,3,4,5,6,category,commodity_transaction,country_or_area,quantity,quantity_footnotes,unit,year
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Exports,Austria,5.0,NaN,"Metric tons, thousand",1996.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Exports,Austria,17.0,NaN,"Metric tons, thousand",1995.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Exports,Belgium,0.0,NaN,"Metric tons, thousand",2014.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Exports,Belgium,0.0,NaN,"Metric tons, thousand",2013.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Exports,Belgium,35.0,NaN,"Metric tons, thousand",2012.0


In [55]:
df_portugal = df[df['country_or_area'] == 'Portugal']
df_portugal

,0,1,2,3,4,5,6,category,commodity_transaction,country_or_area,quantity,quantity_footnotes,unit,year
466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Imports,Portugal,110.0,NaN,"Metric tons, thousand",2014.0
467,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Imports,Portugal,100.0,NaN,"Metric tons, thousand",2013.0
468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Imports,Portugal,105.0,NaN,"Metric tons, thousand",2012.0
469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Imports,Portugal,103.0,NaN,"Metric tons, thousand",2011.0
470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,additives_and_oxygenates,Additives and Oxygenates - Imports,Portugal,122.0,NaN,"Metric tons, thousand",2010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blast_furnace_gas,Blast Furnace Gas - Transformation in electric...,Portugal,1204.0,NaN,Terajoules,1994.0
95759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blast_furnace_gas,Blast Furnace Gas - Transformation in electric...,Portugal,1247.0,NaN,Terajoules,1993.0
95760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blast_furnace_gas,Blast Furnace Gas - Transformation in electric...,Portugal,1378.0,NaN,Terajoules,1992.0
95761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,blast_furnace_gas,Blast Furnace Gas - Transformation in electric...,Portugal,787.0,NaN,Terajoules,1991.0


In [56]:
countries_list = df.drop_duplicates(subset = 'country_or_area')['country_or_area']
countries_list

0                        Austria
2                        Belgium
8                        Czechia
10                       Finland
26                        France
                  ...           
64822         State of Palestine
64941                 Tajikistan
72140    German Dem. R. (former)
91568                   Anguilla
0                            NaN
Name: country_or_area, Length: 206, dtype: object

In [57]:
# countries_list['country_or_area'][1]

In [58]:
# countries_total = []
# for i in range(1,3):
#     country_temp = countries_list[i]
#     df_per_country = df[df['country_or_area'] == str(country_temp)]
#     df_per_country_sum = df_per_country['quantity'].sum()
#     print(df_per_country_sum)